# Convert Keras .h5 Model to TensorFlow SavedModel Format

This notebook shows steps to convert keras model to tensorflow format.

In [10]:
import os

# Load the keras model (Xception) from the url and save it as 'clothing-model-v4.h5'
if not os.path.isfile('clothing-model-v4.h5'):
    !wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model-v4.h5

Convert keras `.h5` to tensorflow `save` format.

In [11]:
import tensorflow as tf
from tensorflow import keras

# Load keras model
model = keras.models.load_model('./clothing-model-v4.h5')

2022-11-29 15:34:34.790584: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12582912 exceeds 10% of free system memory.


In [12]:
model

In [13]:
# Save the model in tensorflow format
tf.saved_model.save(model, 'clothing-model')

INFO:tensorflow:Assets written to: clothing-model/assets


INFO:tensorflow:Assets written to: clothing-model/assets


In [14]:
!ls

clothing-model	      gRFC-tf-serving-connection.ipynb
clothing-model-v4.h5  h5model-to-saved.ipynb


In [15]:
# Take a look what's inside 'clothing-model' folder
!ls clothing-model

assets	saved_model.pb	variables


In [16]:
# !tree clothing-model

Let's take a look what's inside the saved model using the `saved_model_cli` utility that comes with tensorflow.

In [17]:
!saved_model_cli show --dir clothing-model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          input_8: Tensor

**We are interested in the signature, specifically the following one:**

```
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
```

This signature definition has the name `serving_default` that we need to invoke our model, then we have the name of the inputs (`input_8`) that can take arbitrary number of images (hence batch `-1`) of shape `(299 ,299, 3)` and the outputs name (`dense_7`) of arbitrary images of shape (`10`).

We need to take note of the these names:

- `serving_default` - signature
- `input_8` - input
- `dense_7` - output